In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib
!pip install datasets
!pip install -q wandb -U
!pip install -U transformers
!pip install -U accelerate
!pip install trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import json
with open('/content/drive/MyDrive/training_set.json', 'r') as file:
    train_list = json.load(file)
train_list_formatted = []
for i in range(len(train_list)):
  train_list_formatted.append(train_list[i]["text"])

with open('/content/drive/MyDrive/validation_set.json', 'r') as file:
    val_list = json.load(file)
val_list_formatted = []
for i in range(len(val_list)):
  val_list_formatted.append(val_list[i]["text"])

train_dict = {"text": train_list_formatted}
val_dict = {"text": val_list_formatted}



print(train_list_formatted[0])
print(train_list_formatted[1])
print(val_list_formatted[0])
print(val_list_formatted[1])


<s> [INST] How does GIS differ from general data systems? Options: Option A - GIS exclusively uses numerical data, Option B - GIS focuses on georeferenced spatial data, Option C - GIS processes only temporal data, Option D - GIS operates without user interaction. [/INST] The correct answer is Option B. </s>
<s> [INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST] The correct answer is Option B. </s>
<s> [INST] What type of GIS application involves continued spatial data administration? Options: Option A - Institutional GIS applications, Option B - Project-based GIS applications, Option C - Scientific GIS applications, Option D - Temporary GIS applications. [/INST] The correct answer is Option A. </s>
<s> [INST] What is a fundamental problem GIS addresses? Options: Option A - Analyzing static historical data, Option B - Understanding spatial and temporal phen

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)

In [ ]:
print(train_dataset["text"][0])

<s> [INST] How does GIS differ from general data systems? Options: Option A - GIS exclusively uses numerical data, Option B - GIS focuses on georeferenced spatial data, Option C - GIS processes only temporal data, Option D - GIS operates without user interaction. [/INST] The correct answer is Option B. </s>


In [ ]:
type(train_dataset)


datasets.arrow_dataset.Dataset

In [ ]:
from huggingface_hub import notebook_login, login
login(new_session=False,
write_permission=False,
token='',
add_to_git_credential=False)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from transformers import TrainingArguments

import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

import wandb, os
wandb.login()

wandb_project = "gis-qa-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sanskriti-gaekwad (sanskriti-gaekwad-stony-brook-university). Use `wandb login --relogin` to force relogin


In [ ]:
from peft import LoraConfig, get_peft_model


lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    task_type="CAUSAL_LM",
    lora_dropout=0.1
)


model_with_lora = get_peft_model(model, lora_config)


In [ ]:
save_folder_name = "gis_qa_10ep_lr1e-4_r8_alpha32_temp"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/" + save_folder_name,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    logging_steps = 4,
    save_strategy = 'epoch',
    logging_dir = '/content/drive/MyDrive/Colab Notebooks/' + save_folder_name + '/logs',
    evaluation_strategy = 'epoch',
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model_with_lora,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field='text',
    max_seq_length=1024,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will ove

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,2.908000,1.884629
2,1.521200,1.437516
3,1.214900,1.410806
4,1.026200,1.444825
5,0.847300,1.497579
6,0.700900,1.603922
7,0.594300,1.707385
8,0.449300,1.782125
9,0.345900,1.861897
10,0.298700,1.913625


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

TrainOutput(global_step=280, training_loss=0.9570642863001142, metrics={'train_runtime': 668.707, 'train_samples_per_second': 1.675, 'train_steps_per_second': 0.419, 'total_flos': 3904086675750912.0, 'train_loss': 0.9570642863001142, 'epoch': 10.0})

In [ ]:
model_with_lora.save_pretrained("/content/drive/MyDrive/Colab Notebooks/" + save_folder_name + "/mistral-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/" + save_folder_name + "/mistral-finetuned")

('/content/drive/MyDrive/Colab Notebooks/gis_qa_10ep_lr1e-4_nogradientaccumulation/mistral-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/gis_qa_10ep_lr1e-4_nogradientaccumulation/mistral-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/gis_qa_10ep_lr1e-4_nogradientaccumulation/mistral-finetuned/tokenizer.model',
 '/content/drive/MyDrive/Colab Notebooks/gis_qa_10ep_lr1e-4_nogradientaccumulation/mistral-finetuned/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/gis_qa_10ep_lr1e-4_nogradientaccumulation/mistral-finetuned/tokenizer.json')

**Inference**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import pipeline

import torch

import re
from tqdm import tqdm

def extract_correct_answer(inp):
    match = re.search(r"The correct answer is Option ([A-D])", inp)
    if match:
        return match.group(1)
    else:
        return None

def generate_prediction(inp, model, tokenizer):
    return tokenizer.decode( model.generate( tokenizer.encode(inp, return_tensors="pt").to(model.device) )[0], skip_special_tokens=True)


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:

load_folder_name =  "gis_qa_10ep_lr1e-4_r8_alpha32"

model_path = "/content/drive/MyDrive/Colab Notebooks/" + load_folder_name + "/mistral-finetuned"

model_with_lora_trained = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation", model=model_with_lora_trained, tokenizer=tokenizer)

prompt = "[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST]"

output = pipe(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST] The correct answer is Option B. 
Option A refers to data capture, Option C relates


In [ ]:
correct = 0
total = 0

for example in tqdm(val_dataset):

    question = re.search(r"\[INST\](.*?)\[/INST\]", example["text"], re.DOTALL).group(1).strip()

    predicted_option = extract_correct_answer(generate_prediction(question, model_with_lora_trained, tokenizer))

    if predicted_option == extract_correct_answer(example["text"]):
        correct += 1
    total += 1

accuracy = (correct / total) * 100
print(f"Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")


  0%|          | 0/28 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▎         | 1/28 [00:03<01:28,  3.29s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  7%|▋         | 2/28 [00:07<01:33,  3.59s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 11%|█         | 3/28 [00:11<01:40,  4.02s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

Validation Accuracy: 85.71% (24/28)


In [ ]:

load_folder_name =  "gis_qa_10ep_lr3e-4_r8_alpha32"

model_path = "/content/drive/MyDrive/Colab Notebooks/" + load_folder_name + "/mistral-finetuned"

model_with_lora_trained = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation", model=model_with_lora_trained, tokenizer=tokenizer)

prompt = "[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST]"

output = pipe(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST] The correct answer is Option B. 

The correct answer is Option B. 




In [ ]:
correct = 0
total = 0

for example in tqdm(val_dataset):

    question = re.search(r"\[INST\](.*?)\[/INST\]", example["text"], re.DOTALL).group(1).strip()

    predicted_option = extract_correct_answer(generate_prediction(question, model_with_lora_trained, tokenizer))

    if predicted_option == extract_correct_answer(example["text"]):
        correct += 1
    total += 1

accuracy = (correct / total) * 100
print(f"Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")


  0%|          | 0/28 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▎         | 1/28 [00:02<01:02,  2.32s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  7%|▋         | 2/28 [00:04<01:00,  2.33s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 11%|█         | 3/28 [00:07<01:04,  2.60s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

Validation Accuracy: 92.86% (26/28)


In [ ]:

load_folder_name =  "gis_qa_10ep_lr5e-4_r8_alpha32"

model_path = "/content/drive/MyDrive/Colab Notebooks/" + load_folder_name + "/mistral-finetuned"

model_with_lora_trained = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation", model=model_with_lora_trained, tokenizer=tokenizer)

prompt = "[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST]"

output = pipe(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation, Option D - Data presentation. [/INST] The correct answer is Option B. 
The correct answer is Option B. 
The correct


In [ ]:
correct = 0
total = 0

for example in tqdm(val_dataset):

    question = re.search(r"\[INST\](.*?)\[/INST\]", example["text"], re.DOTALL).group(1).strip()

    predicted_option = extract_correct_answer(generate_prediction(question, model_with_lora_trained, tokenizer))

    if predicted_option == extract_correct_answer(example["text"]):
        correct += 1
    total += 1

accuracy = (correct / total) * 100
print(f"Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")


  0%|          | 0/28 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▎         | 1/28 [00:02<01:02,  2.33s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  7%|▋         | 2/28 [00:05<01:08,  2.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 11%|█         | 3/28 [00:07<01:07,  2.69s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

Validation Accuracy: 89.29% (25/28)


**With no training**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import torch

import re
from tqdm import tqdm
from transformers import pipeline

def extract_correct_answer(inp):
    match = re.search(r"Option ([A-D])", inp.split('[/INST]')[1])
    if match:
        return match.group(1)
    else:
        return None


def generate_prediction(inp, model, tokenizer):
    return tokenizer.decode( model.generate( tokenizer.encode(inp, return_tensors="pt").to(model.device) )[0], skip_special_tokens=True)


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from transformers import TrainingArguments

import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation Option D - Data presentation. [/INST]"

output = pipe(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Which of the following is NOT a GIS function? Options: Option A - Data capture, Option B - Data encryption, Option C - Data manipulation Option D - Data presentation. [/INST] Option B - Data encryption is not a GIS function. GIS functions typically involve capturing,


In [ ]:
correct = 0
total = 0

for example in tqdm(val_dataset):

    question = re.search(r"\[INST\](.*?)\[/INST\]", example["text"], re.DOTALL).group(1).strip()

    predicted_option = extract_correct_answer(generate_prediction('[INST] '+ question + ' [/INST]', model, tokenizer))

    if predicted_option == extract_correct_answer(example["text"]):
        correct += 1
    total += 1

accuracy = (correct / total) * 100
print(f"Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")


  0%|          | 0/28 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  4%|▎         | 1/28 [00:03<01:35,  3.54s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  7%|▋         | 2/28 [00:06<01:22,  3.18s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 11%|█         | 3/28 [00:09<01:17,  3.09s/it]The attention mask and the pad token id were not set. As a consequence, you may obs

Validation Accuracy: 89.29% (25/28)
